## Setting up the notebook

In [ ]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install subword-nmt
!pip install sentencepiece
!pip install wandb

  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)


In [ ]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm
import wandb

In [ ]:
# setting up wandb for logging model performance during training
# api key = b5b05b603ec81167154bcdaec184b83d9e96049b
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
source_code = 'eng'
target_code = 'tso'

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
#!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --editable ./

/content/drive/MyDrive/Research/eng-to-tso/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-tso/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 20.3 MB/s eta 0:00:00
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp310-cp310-linux_x86_64.whl size=9581 sha256=b45da998b9bb4c6b754708f1e045168b2101fd34ac2d8e6be52b1e66aa7

## Training NMT model with Byte-Pair Encoding

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe')

In [ ]:
!fairseq-train data-bin \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 100 \
--validate-interval 25 \
--save-interval 25 \
--validate-interval-updates 6000 \
--save-interval-updates 6000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe \
--save-dir checkpoints-bpe \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-tso'

Streaming output truncated to the last 5000 lines.

epoch 054 | valid on 'valid' subset:  26% 63/243 [00:40<01:38,  1.84it/s]2024-10-25 07:23:51 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-10-25 07:23:51 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-10-25 07:23:51 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 054 | valid on 'valid' subset:  26% 64/243 [00:40<01:43,  1.73it/s]
epoch 054 | valid on 'valid' subset:  27% 65/243 [00:41<01:32,  1.92it/s]2024-10-25 07:23:52 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-10-25 07:23:52 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-10-25 07:23:52 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this m

## Training NMT with Unigram Language Model

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm')

In [ ]:
!fairseq-train data-bin \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 100 \
--validate-interval 25 \
--save-interval 25 \
--validate-interval-updates 6000 \
--save-interval-updates 6000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe sentencepiece \
--save-dir checkpoints-ulm \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-tso'

Streaming output truncated to the last 5000 lines.
2024-10-25 09:49:05 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 054 | valid on 'valid' subset:  32% 79/246 [00:50<02:02,  1.36it/s]2024-10-25 09:49:06 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-10-25 09:49:06 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-10-25 09:49:06 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 054 | valid on 'valid' subset:  33% 80/246 [00:51<01:54,  1.45it/s]2024-10-25 09:49:07 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-10-25 09:49:07 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-10-25 09:49:07 | WARNING | sacrebl

## Training NMT with BPE-Dropout

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP')

In [ ]:
!fairseq-train data-bin-25 \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 4 \
--validate-interval 1 \
--save-interval 1 \
--validate-interval-updates 6000 \
--save-interval-updates 6000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe \
--save-dir checkpoints-bpeDROP \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-tso'

Streaming output truncated to the last 5000 lines.
epoch 002 | valid on 'valid' subset:  76% 185/243 [02:04<00:44,  1.30it/s]
epoch 002 | valid on 'valid' subset:  77% 186/243 [02:04<00:42,  1.34it/s]
epoch 002 | valid on 'valid' subset:  77% 187/243 [02:05<00:42,  1.33it/s]
epoch 002 | valid on 'valid' subset:  77% 188/243 [02:06<00:43,  1.25it/s]
epoch 002 | valid on 'valid' subset:  78% 189/243 [02:07<00:38,  1.40it/s]
epoch 002 | valid on 'valid' subset:  78% 190/243 [02:07<00:35,  1.47it/s]
epoch 002 | valid on 'valid' subset:  79% 191/243 [02:08<00:33,  1.54it/s]
epoch 002 | valid on 'valid' subset:  79% 192/243 [02:08<00:30,  1.70it/s]
epoch 002 | valid on 'valid' subset:  79% 193/243 [02:09<00:29,  1.70it/s]
epoch 002 | valid on 'valid' subset:  80% 194/243 [02:09<00:28,  1.70it/s]
epoch 002 | valid on 'valid' subset:  80% 195/243 [02:10<00:26,  1.78it/s]
epoch 002 | valid on 'valid' subset:  81% 196/243 [02:11<00:26,  1.74it/s]
epoch 002 | valid on 'valid' subset:  81% 197/243

## Training NMT with ULM with Subword Regularization

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR')

In [ ]:
!fairseq-train data-bin-25 \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 4 \
--validate-interval 1 \
--save-interval 1 \
--validate-interval-updates 6000 \
--save-interval-updates 6000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe sentencepiece \
--save-dir checkpoints-ulmSR \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-tso'

Streaming output truncated to the last 5000 lines.
epoch 003 | valid on 'valid' subset:  93% 230/246 [02:44<00:13,  1.19it/s]
epoch 003 | valid on 'valid' subset:  94% 231/246 [02:45<00:13,  1.14it/s]
epoch 003 | valid on 'valid' subset:  94% 232/246 [02:46<00:13,  1.08it/s]
epoch 003 | valid on 'valid' subset:  95% 233/246 [02:48<00:12,  1.03it/s]
epoch 003 | valid on 'valid' subset:  95% 234/246 [02:49<00:12,  1.00s/it]
epoch 003 | valid on 'valid' subset:  96% 235/246 [02:49<00:10,  1.06it/s]
epoch 003 | valid on 'valid' subset:  96% 236/246 [02:50<00:08,  1.15it/s]
epoch 003 | valid on 'valid' subset:  96% 237/246 [02:51<00:07,  1.21it/s]
epoch 003 | valid on 'valid' subset:  97% 238/246 [02:52<00:06,  1.28it/s]
epoch 003 | valid on 'valid' subset:  97% 239/246 [02:52<00:05,  1.28it/s]
epoch 003 | valid on 'valid' subset:  98% 240/246 [02:53<00:04,  1.30it/s]
epoch 003 | valid on 'valid' subset:  98% 241/246 [02:54<00:03,  1.29it/s]
epoch 003 | valid on 'valid' subset:  98% 242/246

## Training NMT with Task-Specific Tokenizer

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok')

In [ ]:
!fairseq-train data-bin-nmt \
--arch transformer \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion 'label_smoothed_cross_entropy' \
--label-smoothing 0.1 \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
--max-epoch 100 \
--validate-interval 25 \
--save-interval 25 \
--validate-interval-updates 6000 \
--save-interval-updates 6000 \
--log-interval 100 \
--curriculum 0 \
--no-epoch-checkpoints \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 0, "max_len_b": 100, "len_pen": 1}' \
--eval-bleu-detok space \
--eval-bleu-remove-bpe sentencepiece \
--save-dir checkpoints-task-nmt \
--ddp-backend=no_c10d \
--wandb-project 'fairseq-standard-subword-tok-eng-to-tso'

Streaming output truncated to the last 5000 lines.
2024-11-06 08:28:18 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 054 | valid on 'valid' subset:  32% 79/246 [00:59<02:12,  1.26it/s]2024-11-06 08:28:19 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-06 08:28:19 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-06 08:28:19 | WARNING | sacrebleu | If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.

epoch 054 | valid on 'valid' subset:  33% 80/246 [01:00<02:22,  1.16it/s]2024-11-06 08:28:20 | WARNING | sacrebleu | That's 100 lines that end in a tokenized period ('.')
2024-11-06 08:28:20 | WARNING | sacrebleu | It looks like you forgot to detokenize your test data, which may hurt your score.
2024-11-06 08:28:20 | WARNING | sacrebl